In [6]:
from model import *
from data import *
import os
import datetime
from keras.callbacks import ModelCheckpoint

In [2]:
input_dir = "data/train/image"
target_dir = "data/train/label"


input_img_paths, target_img_paths = get_train_data_paths(input_dir, target_dir)

print(input_img_paths)
print(target_img_paths)


['data/train/image/0.png', 'data/train/image/1.png', 'data/train/image/10.png', 'data/train/image/11.png', 'data/train/image/12.png', 'data/train/image/13.png', 'data/train/image/14.png', 'data/train/image/15.png', 'data/train/image/16.png', 'data/train/image/17.png', 'data/train/image/18.png', 'data/train/image/19.png', 'data/train/image/2.png', 'data/train/image/20.png', 'data/train/image/21.png', 'data/train/image/22.png', 'data/train/image/23.png', 'data/train/image/24.png', 'data/train/image/25.png', 'data/train/image/26.png', 'data/train/image/27.png', 'data/train/image/28.png', 'data/train/image/29.png', 'data/train/image/3.png', 'data/train/image/4.png', 'data/train/image/5.png', 'data/train/image/6.png', 'data/train/image/7.png', 'data/train/image/8.png', 'data/train/image/9.png']
['data/train/label/0.png', 'data/train/label/1.png', 'data/train/label/10.png', 'data/train/label/11.png', 'data/train/label/12.png', 'data/train/label/13.png', 'data/train/label/14.png', 'data/train

In [4]:
# Load data
train_datagen = DataLoader(batch_size = 1, 
                           img_size = (512,512), 
                           input_img_paths = input_img_paths,
                           target_img_paths = target_img_paths,
                            rescale = 1 / 255)

print(train_datagen)

In [ ]:

# Load and train model
model = unet_model()

callbacks = []
weights_path = 'unet_membrane' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.hdf5'
checkpoint = ModelCheckpoint(weights_path, monitor='loss', verbose=1, save_best_only=True)
callbacks.append(checkpoint)
model.fit(train_datagen, steps_per_epoch=30, epochs=10, verbose=1, callbacks = callbacks)

Model: "unet_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 512, 512, 64) 640         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 512, 512, 64) 36928       conv2d_49[0][0]                  
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 512, 512, 64) 256         conv2d_50[0][0]                  
_________________________________________________________________________________________

Epoch 1/10


In [ ]:
# Upload weights to s3
import logging
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket = 'em-cells'
object_name = 'weights/' + weights_path

upload_file(weights_path, bucket, object_name)
